# Style-Based Rotation Strategy

## Overview
This notebook implements style-focused rotation strategies using ratio-based mean reversion signals.

### Strategy Focus:
- **Growth vs Value (IWF/IWD)**: Classic style rotation based on market cycles
- **Small Cap vs Large Cap (IWM/SPY)**: Size-based rotation driven by risk appetite

### Key Concepts:
- **Style Cycles**: Growth outperforms in bull markets, Value in bear/recovery
- **Size Premium**: Small caps lead in early cycle, Large caps in late cycle
- **Interest Rate Sensitivity**: Growth sensitive to rates, Value less so
- **Economic Phases**: Early/mid cycle (Small/Growth) vs Late cycle (Large/Value)

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

print("📊 Style-Based Rotation Strategy Environment Ready")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("🎯 Strategy: Growth/Value + Size-Based Rotations")

📊 Style-Based Rotation Strategy Environment Ready
📅 Analysis Date: 2025-09-01 16:16
🎯 Strategy: Growth/Value + Size-Based Rotations


## 1. Style Factor ETF Data Collection

In [4]:
# Define style-based rotation pairs
STYLE_FACTORS = {
    # Growth vs Value (Russell)
    'IWF': 'IWF',    # iShares Russell 1000 Growth
    'IWD': 'IWD',    # iShares Russell 1000 Value
    
    # Growth vs Value (Vanguard)
    'VUG': 'VUG',    # Vanguard Growth ETF
    'VTV': 'VTV',    # Vanguard Value ETF
    
    # Size factors
    'IWM': 'IWM',    # iShares Russell 2000 (Small Cap)
    'SPY': 'SPY',    # S&P 500 (Large Cap)
    'IJH': 'IJH',    # iShares Core S&P Mid-Cap
    
    # Quality and Momentum factors
    'QUAL': 'QUAL',  # iShares Edge MSCI USA Quality Factor
    'MTUM': 'MTUM',  # iShares Edge MSCI USA Momentum Factor
    
    # Context ETFs
    'TLT': 'TLT',    # 20+ Year Treasury (rates)
    'VIX': '^VIX'    # Volatility index
}

def fetch_style_factor_data(etfs_dict, period='5y'):
    """Fetch style factor ETF data with error handling"""
    print("🔄 Fetching style factor ETF data...")
    
    style_data = {}
    for etf_name, symbol in etfs_dict.items():
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period, interval='1d')
            if not data.empty and len(data) > 100:
                style_data[etf_name] = data['Close']
                print(f"  ✅ {etf_name}: {len(data)} days")
        except Exception as e:
            print(f"  ❌ {etf_name}: Failed")
    
    if style_data:
        df = pd.DataFrame(style_data).dropna()
        print(f"\n✅ Style factor dataset: {len(df.columns)} ETFs, {len(df)} observations")
        print(f"📊 Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
        return df
    return pd.DataFrame()

# Fetch style factor data
style_prices = fetch_style_factor_data(STYLE_FACTORS)
print(f"\n📈 Available Style Factor ETFs: {list(style_prices.columns)}")
style_prices.head()

🔄 Fetching style factor ETF data...
  ✅ IWF: 1256 days
  ✅ IWD: 1256 days
  ✅ VUG: 1256 days
  ✅ VTV: 1256 days
  ✅ IWM: 1256 days
  ✅ SPY: 1256 days
  ✅ IJH: 1256 days
  ✅ QUAL: 1256 days
  ✅ MTUM: 1256 days
  ✅ TLT: 1256 days
  ✅ VIX: 1256 days

✅ Style factor dataset: 11 ETFs, 0 observations


IndexError: index 0 is out of bounds for axis 0 with size 0

## 2. Style-Based Rotation Strategy Implementation

In [ ]:
def calculate_style_ratio(price_data, etf1, etf2):
    """Calculate style factor ratio between two ETFs"""
    ratio = price_data[etf1] / price_data[etf2]
    return ratio

def calculate_style_zscore(ratio, lookback_window=60):
    """Calculate rolling z-score of style ratio with cycle context"""
    rolling_mean = ratio.rolling(window=lookback_window).mean()
    rolling_std = ratio.rolling(window=lookback_window).std()
    z_score = (ratio - rolling_mean) / rolling_std
    return z_score, rolling_mean, rolling_std

def generate_style_rotation_signals(price_data, etf1, etf2, 
                                  entry_threshold=2.0, exit_threshold=0.5, lookback_window=60):
    """Generate style-based rotation signals with economic context"""
    
    # Calculate ratio and z-score
    ratio = calculate_style_ratio(price_data, etf1, etf2)
    z_score, rolling_mean, rolling_std = calculate_style_zscore(ratio, lookback_window)
    
    # Create signals DataFrame
    signals = pd.DataFrame(index=price_data.index)
    signals['ratio'] = ratio
    signals['z_score'] = z_score
    signals['rolling_mean'] = rolling_mean
    signals['rolling_std'] = rolling_std
    
    # Add interest rate context if available
    if 'TLT' in price_data.columns:
        signals['rate_environment'] = -price_data['TLT'].pct_change(20)  # Negative TLT change = rising rates
        tlt_ma = price_data['TLT'].rolling(60).mean()
        signals['rate_trend'] = (price_data['TLT'] - tlt_ma) / tlt_ma  # TLT vs MA
    
    # Add volatility/risk context if VIX available
    if 'VIX' in price_data.columns:
        signals['market_stress'] = price_data['VIX'].rolling(10).mean()
        signals['risk_regime'] = (price_data['VIX'] > 25).astype(int)  # High vol regime
    
    # Style rotation signals
    signals['rotate_to_etf1'] = z_score < -entry_threshold   # Ratio low, rotate to ETF1
    signals['rotate_to_etf2'] = z_score > entry_threshold    # Ratio high, rotate to ETF2
    
    # Enhanced signals with rate/risk context
    if 'rate_environment' in signals.columns and 'market_stress' in signals.columns:
        # Strengthen growth signals in low rate/low vol environment
        growth_boost = (signals['rate_environment'] < 0) & (signals['market_stress'] < 20)
        # Strengthen value signals in high rate/high vol environment
        value_boost = (signals['rate_environment'] > 0) & (signals['market_stress'] > 25)
        
        if 'Growth' in etf1 or etf1 in ['IWF', 'VUG']:
            signals['enhanced_growth_signal'] = signals['rotate_to_etf1'] | (growth_boost & (z_score < -1.5))
        if 'Value' in etf2 or etf2 in ['IWD', 'VTV']:
            signals['enhanced_value_signal'] = signals['rotate_to_etf2'] | (value_boost & (z_score > 1.5))
    
    # Neutral/exit signals
    signals['neutral_signal'] = abs(z_score) < exit_threshold
    
    # Position logic for style allocation
    signals['style_allocation'] = 0  # 0 = neutral, 1 = favor ETF1, -1 = favor ETF2
    current_allocation = 0
    
    for i in range(len(signals)):
        if current_allocation == 0:  # Neutral allocation
            if signals['rotate_to_etf1'].iloc[i]:
                current_allocation = 1  # Rotate to ETF1
            elif signals['rotate_to_etf2'].iloc[i]:
                current_allocation = -1  # Rotate to ETF2
        elif current_allocation != 0:  # Have style bias
            if signals['neutral_signal'].iloc[i]:
                current_allocation = 0  # Back to neutral
        
        signals['style_allocation'].iloc[i] = current_allocation
    
    # Calculate strategy returns
    signals['etf1_return'] = price_data[etf1].pct_change()
    signals['etf2_return'] = price_data[etf2].pct_change()
    
    # Strategy return based on style allocation
    strategy_returns = []
    for i in range(len(signals)):
        allocation = signals['style_allocation'].iloc[i]
        etf1_ret = signals['etf1_return'].iloc[i]
        etf2_ret = signals['etf2_return'].iloc[i]
        
        if allocation == 1:  # Favor ETF1
            strategy_ret = etf1_ret
        elif allocation == -1:  # Favor ETF2
            strategy_ret = etf2_ret
        else:  # Neutral - equal weight
            strategy_ret = 0.5 * etf1_ret + 0.5 * etf2_ret
        
        strategy_returns.append(strategy_ret if not np.isnan(strategy_ret) else 0)
    
    signals['strategy_return'] = strategy_returns
    signals['cumulative_return'] = (1 + pd.Series(strategy_returns, index=signals.index)).cumprod()
    
    # Buy and hold benchmarks
    signals['etf1_cumulative'] = (1 + signals['etf1_return'].fillna(0)).cumprod()
    signals['etf2_cumulative'] = (1 + signals['etf2_return'].fillna(0)).cumprod()
    signals['equal_weight_cumulative'] = (1 + (0.5 * signals['etf1_return'] + 0.5 * signals['etf2_return']).fillna(0)).cumprod()
    
    return signals

print("✅ Style-based rotation strategy functions implemented")

## 3. Test Style-Based Rotation Strategies

In [ ]:
# Test style-based rotation strategies
print("🎯 Testing Style-Based Rotation Strategies\n")

# Define style rotation pairs with descriptions
STYLE_ROTATION_PAIRS = [
    ('IWF', 'IWD', 'Growth vs Value (Russell Growth vs Value)'),
    ('VUG', 'VTV', 'Growth vs Value (Vanguard Growth vs Value)'),
    ('IWM', 'SPY', 'Small Cap vs Large Cap (Russell 2000 vs S&P 500)')
]

style_rotation_results = []

for etf1, etf2, description in STYLE_ROTATION_PAIRS:
    if etf1 in style_prices.columns and etf2 in style_prices.columns:
        try:
            print(f"📊 Analyzing {description}...")
            
            # Generate rotation signals
            signals = generate_style_rotation_signals(style_prices, etf1, etf2)
            
            # Calculate performance metrics
            strategy_cumulative = signals['cumulative_return']
            total_return = strategy_cumulative.iloc[-1] - 1
            
            # Benchmark returns
            etf1_total = signals['etf1_cumulative'].iloc[-1] - 1
            etf2_total = signals['etf2_cumulative'].iloc[-1] - 1
            equal_weight_total = signals['equal_weight_cumulative'].iloc[-1] - 1
            
            # Risk metrics
            volatility = signals['strategy_return'].std() * np.sqrt(252)
            sharpe_ratio = (total_return * 252 / len(signals)) / volatility if volatility > 0 else 0
            
            max_drawdown = ((strategy_cumulative / strategy_cumulative.expanding().max()) - 1).min()
            
            # Rotation statistics
            rotations = signals['style_allocation'].diff().abs().sum() / 2
            time_in_etf1 = (signals['style_allocation'] == 1).mean()
            time_in_etf2 = (signals['style_allocation'] == -1).mean()
            time_neutral = (signals['style_allocation'] == 0).mean()
            
            # Current status
            current_ratio = signals['ratio'].iloc[-1]
            current_zscore = signals['z_score'].iloc[-1]
            current_allocation = signals['style_allocation'].iloc[-1]
            
            # Style-specific metrics
            rate_environment = 0
            market_stress = 0
            
            if 'rate_environment' in signals.columns:
                rate_environment = signals['rate_environment'].iloc[-1]
            if 'market_stress' in signals.columns:
                market_stress = signals['market_stress'].iloc[-1]
            
            recent_volatility = signals['ratio'].rolling(30).std().iloc[-1]
            trend_strength = abs(signals['z_score'].rolling(20).mean().iloc[-1])
            
            # Style cycle analysis
            cycle_persistence = signals['style_allocation'].groupby((signals['style_allocation'] != signals['style_allocation'].shift()).cumsum()).size().mean()
            
            style_rotation_results.append({
                'strategy': description,
                'etf1': etf1,
                'etf2': etf2,
                'strategy_return': total_return,
                'etf1_return': etf1_total,
                'etf2_return': etf2_total,
                'equal_weight_return': equal_weight_total,
                'outperformance': total_return - equal_weight_total,
                'sharpe_ratio': sharpe_ratio,
                'max_drawdown': max_drawdown,
                'volatility': volatility,
                'num_rotations': rotations,
                'time_in_etf1': time_in_etf1,
                'time_in_etf2': time_in_etf2,
                'time_neutral': time_neutral,
                'current_ratio': current_ratio,
                'current_zscore': current_zscore,
                'current_allocation': current_allocation,
                'rate_environment': rate_environment,
                'market_stress': market_stress,
                'recent_volatility': recent_volatility,
                'trend_strength': trend_strength,
                'cycle_persistence': cycle_persistence
            })
            
            print(f"  Strategy Return: {total_return:.2%}")
            print(f"  {etf1} Return: {etf1_total:.2%}")
            print(f"  {etf2} Return: {etf2_total:.2%}")
            print(f"  Equal Weight: {equal_weight_total:.2%}")
            print(f"  Outperformance: {total_return - equal_weight_total:.2%}")
            print(f"  Sharpe Ratio: {sharpe_ratio:.3f}")
            print(f"  Current Z-Score: {current_zscore:.2f}")
            print(f"  Style Cycle Persistence: {cycle_persistence:.1f} days")
            if rate_environment != 0:
                print(f"  Rate Environment: {rate_environment:.2%}")
            print()
            
        except Exception as e:
            print(f"  ❌ Error: {str(e)}\n")

# Results analysis
if style_rotation_results:
    style_rotation_df = pd.DataFrame(style_rotation_results)
    
    print(f"📊 Style-Based Rotation Results Summary:")
    display(style_rotation_df[['strategy', 'strategy_return', 'outperformance', 'sharpe_ratio', 'max_drawdown', 'num_rotations', 'cycle_persistence']].round(3))
    
    # Best performing style strategy
    best_style_strategy = style_rotation_df.loc[style_rotation_df['sharpe_ratio'].idxmax()]
    print(f"\n🏆 Best Style Strategy: {best_style_strategy['strategy']}")
    print(f"   Sharpe Ratio: {best_style_strategy['sharpe_ratio']:.3f}")
    print(f"   Outperformance: {best_style_strategy['outperformance']:.2%}")
    print(f"   Style Cycle Persistence: {best_style_strategy['cycle_persistence']:.1f} days")
    
else:
    print("❌ No style-based rotation results generated")

## 4. Detailed Analysis and Visualization

In [ ]:
def plot_style_rotation_analysis(signals, etf1, etf2, strategy_name):
    """Comprehensive visualization of style-based rotation strategy"""
    
    fig, axes = plt.subplots(6, 1, figsize=(15, 26))
    
    # 1. Style Factor Ratio with Bands
    ax1 = axes[0]
    ax1.plot(signals.index, signals['ratio'], label=f'{etf1}/{etf2} Ratio', color='darkviolet', linewidth=1.5)
    ax1.plot(signals.index, signals['rolling_mean'], label='Rolling Mean', color='red', linestyle='--')
    ax1.fill_between(signals.index, 
                     signals['rolling_mean'] + 2*signals['rolling_std'],
                     signals['rolling_mean'] - 2*signals['rolling_std'],
                     alpha=0.2, color='mediumpurple', label='±2σ Band')
    ax1.set_title(f'{strategy_name}: {etf1}/{etf2} Ratio Analysis', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Ratio Value')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Z-Score with Style Rotation Thresholds
    ax2 = axes[1]
    ax2.plot(signals.index, signals['z_score'], label='Z-Score', color='indigo', linewidth=1.5)
    ax2.axhline(y=2, color='red', linestyle='--', label=f'Rotate to {etf2} (+2σ)')
    ax2.axhline(y=-2, color='green', linestyle='--', label=f'Rotate to {etf1} (-2σ)')
    ax2.axhline(y=0.5, color='orange', linestyle=':', label='Neutral Threshold')
    ax2.axhline(y=-0.5, color='orange', linestyle=':')
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    
    # Highlight extreme style cycles
    extreme_etf1 = signals['z_score'] < -2.5
    extreme_etf2 = signals['z_score'] > 2.5
    ax2.fill_between(signals.index, -4, 4, where=extreme_etf1, alpha=0.1, color='green', label=f'Strong {etf1} Cycle')
    ax2.fill_between(signals.index, -4, 4, where=extreme_etf2, alpha=0.1, color='red', label=f'Strong {etf2} Cycle')
    
    ax2.set_title('Z-Score with Style Rotation Thresholds', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Z-Score')
    ax2.set_ylim(-4, 4)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Interest Rate Environment (if available)
    ax3 = axes[2]
    if 'rate_environment' in signals.columns:
        ax3.plot(signals.index, signals['rate_environment'], label='Rate Environment (20d)', color='brown', linewidth=1.5)
        ax3.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        ax3.axhline(y=0.02, color='red', linestyle='--', alpha=0.7, label='Rising Rates (Favor Value)')
        ax3.axhline(y=-0.02, color='green', linestyle='--', alpha=0.7, label='Falling Rates (Favor Growth)')
        
        # Overlay Market Stress if available
        if 'market_stress' in signals.columns:
            ax3_twin = ax3.twinx()
            ax3_twin.plot(signals.index, signals['market_stress'], label='Market Stress (VIX)', color='red', alpha=0.7)
            ax3_twin.axhline(y=25, color='red', linestyle=':', alpha=0.5, label='High Stress')
            ax3_twin.set_ylabel('VIX Level', color='red')
            ax3_twin.legend(loc='upper right')
        
        ax3.set_title('Interest Rate and Market Stress Context', fontsize=14, fontweight='bold')
    else:
        ax3.text(0.5, 0.5, 'Rate/Stress Context Not Available', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Rate/Stress Context (Not Available)', fontsize=14, fontweight='bold')
    
    ax3.set_ylabel('Rate Environment')
    ax3.legend(loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # 4. Style Allocation Over Time
    ax4 = axes[3]
    
    etf1_periods = signals['style_allocation'] == 1
    etf2_periods = signals['style_allocation'] == -1
    neutral_periods = signals['style_allocation'] == 0
    
    ax4.fill_between(signals.index, 0, 1, where=etf1_periods, 
                    alpha=0.7, color='darkgreen', label=f'Favor {etf1}')
    ax4.fill_between(signals.index, 0, -1, where=etf2_periods, 
                    alpha=0.7, color='darkred', label=f'Favor {etf2}')
    ax4.fill_between(signals.index, -0.1, 0.1, where=neutral_periods, 
                    alpha=0.5, color='gray', label='Neutral')
    
    ax4.set_title('Style Factor Allocation Over Time', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Allocation')
    ax4.set_ylim(-1.2, 1.2)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Individual Style Performance
    ax5 = axes[4]
    ax5.plot(signals.index, signals['etf1_cumulative'], label=f'{etf1}', color='blue', linewidth=2)
    ax5.plot(signals.index, signals['etf2_cumulative'], label=f'{etf2}', color='red', linewidth=2)
    ax5.plot(signals.index, signals['equal_weight_cumulative'], label='Equal Weight', color='gray', linestyle='--', linewidth=2)
    
    # Add economic cycle context if VIX available
    if 'VIX' in style_prices.columns:
        bear_markets = style_prices['VIX'] > 30  # High VIX = bear market periods
        if bear_markets.any():
            ax5.fill_between(signals.index, 0, signals['etf1_cumulative'].max(), 
                            where=bear_markets, alpha=0.1, color='red', label='Bear Market Periods')
    
    ax5.set_title('Individual Style Factor Performance', fontsize=14, fontweight='bold')
    ax5.set_ylabel('Cumulative Return')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Strategy vs Benchmark Performance
    ax6 = axes[5]
    ax6.plot(signals.index, signals['cumulative_return'], label='Style Rotation Strategy', color='darkviolet', linewidth=3)
    ax6.plot(signals.index, signals['equal_weight_cumulative'], label='Equal Weight Benchmark', color='black', linestyle='--', linewidth=2)
    
    # Add performance statistics
    total_return = signals['cumulative_return'].iloc[-1] - 1
    benchmark_return = signals['equal_weight_cumulative'].iloc[-1] - 1
    outperformance = total_return - benchmark_return
    
    ax6.text(0.02, 0.98, f'Strategy Return: {total_return:.1%}\nBenchmark Return: {benchmark_return:.1%}\nOutperformance: {outperformance:.1%}', 
            transform=ax6.transAxes, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.8))
    
    ax6.set_title(f'{strategy_name} Performance vs Benchmark', fontsize=14, fontweight='bold')
    ax6.set_ylabel('Cumulative Return')
    ax6.set_xlabel('Date')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Generate detailed analysis for each style strategy
if 'style_rotation_results' in locals() and style_rotation_results:
    for result in style_rotation_results:
        etf1, etf2 = result['etf1'], result['etf2']
        strategy_name = result['strategy']
        
        print(f"\n📊 Detailed Analysis: {strategy_name}")
        
        # Generate signals for visualization
        signals = generate_style_rotation_signals(style_prices, etf1, etf2)
        
        # Plot analysis
        plot_style_rotation_analysis(signals, etf1, etf2, strategy_name)
        
        # Performance and style metrics
        print(f"\n📈 Performance & Style Metrics:")
        print(f"   Strategy Return: {result['strategy_return']:.2%}")
        print(f"   Outperformance: {result['outperformance']:.2%}")
        print(f"   Sharpe Ratio: {result['sharpe_ratio']:.3f}")
        print(f"   Max Drawdown: {result['max_drawdown']:.2%}")
        print(f"   Volatility: {result['volatility']:.2%}")
        print(f"   Number of Rotations: {result['num_rotations']:.0f}")
        print(f"   Style Cycle Persistence: {result['cycle_persistence']:.1f} days")
        
        print(f"\n⏱️ Style Factor Time Allocation:")
        print(f"   Time in {etf1}: {result['time_in_etf1']:.1%}")
        print(f"   Time in {etf2}: {result['time_in_etf2']:.1%}")
        print(f"   Time Neutral: {result['time_neutral']:.1%}")
        
        print(f"\n📊 Economic Context:")
        if result['rate_environment'] != 0:
            rate_desc = "Rising" if result['rate_environment'] > 0 else "Falling"
            print(f"   Rate Environment: {rate_desc} ({result['rate_environment']:.2%})")
        if result['market_stress'] != 0:
            stress_desc = "High" if result['market_stress'] > 25 else "Moderate" if result['market_stress'] > 15 else "Low"
            print(f"   Market Stress: {stress_desc} (VIX: {result['market_stress']:.1f})")
        
        print(f"\n📍 Current Style Status:")
        current_allocation = result['current_allocation']
        if current_allocation == 1:
            allocation_text = f"Favor {etf1}"
        elif current_allocation == -1:
            allocation_text = f"Favor {etf2}"
        else:
            allocation_text = "Neutral Style Allocation"
        
        print(f"   Current Allocation: {allocation_text}")
        print(f"   Current Z-Score: {result['current_zscore']:.2f}")
        
        # Style cycle interpretation
        if result['current_zscore'] < -2:
            style_phase = f"{etf1} Dominance Phase"
        elif result['current_zscore'] > 2:
            style_phase = f"{etf2} Dominance Phase"
        elif abs(result['current_zscore']) < 0.5:
            style_phase = "Balanced Style Phase"
        else:
            style_phase = "Transitional Style Phase"
        
        print(f"   Style Cycle Phase: {style_phase}")
        print("\n" + "="*80)

## 5. Style Strategy Optimization

In [ ]:
def optimize_style_rotation_strategy(price_data, etf1, etf2, strategy_name):
    """Optimize style-based rotation strategy parameters"""
    print(f"🔧 Optimizing {strategy_name} Strategy...\n")
    
    # Parameter ranges for style optimization
    entry_thresholds = [1.5, 2.0, 2.5, 3.0]
    exit_thresholds = [0.2, 0.5, 0.8, 1.0]
    lookback_windows = [30, 60, 90, 120]  # Different style cycle lengths
    
    optimization_results = []
    
    for entry_thresh in entry_thresholds:
        for exit_thresh in exit_thresholds:
            for lookback in lookback_windows:
                try:
                    # Generate signals with test parameters
                    test_signals = generate_style_rotation_signals(
                        price_data, etf1, etf2,
                        entry_threshold=entry_thresh,
                        exit_threshold=exit_thresh,
                        lookback_window=lookback
                    )
                    
                    # Calculate performance metrics
                    strategy_return = test_signals['cumulative_return'].iloc[-1] - 1
                    equal_weight_return = test_signals['equal_weight_cumulative'].iloc[-1] - 1
                    outperformance = strategy_return - equal_weight_return
                    
                    volatility = test_signals['strategy_return'].std() * np.sqrt(252)
                    sharpe_ratio = (strategy_return * 252 / len(test_signals)) / volatility if volatility > 0 else 0
                    
                    max_drawdown = ((test_signals['cumulative_return'] / test_signals['cumulative_return'].expanding().max()) - 1).min()
                    num_rotations = test_signals['style_allocation'].diff().abs().sum() / 2
                    
                    # Style-specific metrics
                    etf1_time = (test_signals['style_allocation'] == 1).mean()
                    etf2_time = (test_signals['style_allocation'] == -1).mean()
                    cycle_persistence = test_signals['style_allocation'].groupby((test_signals['style_allocation'] != test_signals['style_allocation'].shift()).cumsum()).size().mean()
                    
                    optimization_results.append({
                        'entry_threshold': entry_thresh,
                        'exit_threshold': exit_thresh,
                        'lookback_window': lookback,
                        'strategy_return': strategy_return,
                        'outperformance': outperformance,
                        'sharpe_ratio': sharpe_ratio,
                        'max_drawdown': max_drawdown,
                        'num_rotations': num_rotations,
                        'etf1_time': etf1_time,
                        'etf2_time': etf2_time,
                        'cycle_persistence': cycle_persistence
                    })
                    
                except Exception as e:
                    continue
    
    if optimization_results:
        opt_df = pd.DataFrame(optimization_results)
        
        # Find best parameters by Sharpe ratio
        best_params = opt_df.loc[opt_df['sharpe_ratio'].idxmax()]
        
        print("🏆 Optimal Parameters (by Sharpe Ratio):")
        print(f"   Entry Threshold: {best_params['entry_threshold']}")
        print(f"   Exit Threshold: {best_params['exit_threshold']}")
        print(f"   Lookback Window: {best_params['lookback_window']} days")
        print(f"   Sharpe Ratio: {best_params['sharpe_ratio']:.3f}")
        print(f"   Strategy Return: {best_params['strategy_return']:.2%}")
        print(f"   Outperformance: {best_params['outperformance']:.2%}")
        print(f"   Cycle Persistence: {best_params['cycle_persistence']:.1f} days")
        
        # Alternative optimization by cycle persistence (longer cycles may be better)
        best_persistence = opt_df.loc[opt_df['cycle_persistence'].idxmax()]
        print(f"\n🎯 Best Cycle Persistence Parameters:")
        print(f"   Entry: {best_persistence['entry_threshold']}, Exit: {best_persistence['exit_threshold']}, Lookback: {best_persistence['lookback_window']}")
        print(f"   Cycle Persistence: {best_persistence['cycle_persistence']:.1f} days")
        print(f"   Sharpe: {best_persistence['sharpe_ratio']:.3f}")
        
        # Show top parameter combinations
        print(f"\n📊 Top 10 Parameter Combinations (by Sharpe):")
        top_params = opt_df.nlargest(10, 'sharpe_ratio')[['entry_threshold', 'exit_threshold', 'lookback_window', 'sharpe_ratio', 'outperformance', 'cycle_persistence']]
        display(top_params.round(3))
        
        return best_params, opt_df
    
    return None, None

# Optimize each style strategy
style_optimization_summary = []

if 'style_rotation_results' in locals() and style_rotation_results:
    for result in style_rotation_results:
        etf1, etf2 = result['etf1'], result['etf2']
        strategy_name = result['strategy']
        
        best_params, opt_df = optimize_style_rotation_strategy(style_prices, etf1, etf2, strategy_name)
        
        if best_params is not None:
            style_optimization_summary.append({
                'strategy': strategy_name,
                'etf1': etf1,
                'etf2': etf2,
                'best_entry_threshold': best_params['entry_threshold'],
                'best_exit_threshold': best_params['exit_threshold'],
                'best_lookback': best_params['lookback_window'],
                'optimized_sharpe': best_params['sharpe_ratio'],
                'optimized_return': best_params['strategy_return'],
                'optimized_outperformance': best_params['outperformance'],
                'optimal_cycle_persistence': best_params['cycle_persistence']
            })
        
        print("\n" + "="*80 + "\n")
    
    # Summary of optimized style strategies
    if style_optimization_summary:
        style_opt_summary_df = pd.DataFrame(style_optimization_summary)
        print("📊 Style Strategy Optimization Summary:")
        display(style_opt_summary_df.round(3))

## 6. Current Style Trading Signals

In [ ]:
def get_current_style_signals(style_rotation_results):
    """Get current style-based rotation signals with economic context"""
    print("🎯 Current Style-Based Rotation Signals\n")
    
    current_signals = []
    
    for result in style_rotation_results:
        strategy_name = result['strategy']
        current_zscore = result['current_zscore']
        current_allocation = result['current_allocation']
        etf1, etf2 = result['etf1'], result['etf2']
        rate_environment = result['rate_environment']
        market_stress = result['market_stress']
        
        # Determine style signal strength and recommendation
        if current_zscore < -2.5:
            signal = f"STRONG {etf1} DOMINANCE"
            strength = "Very Strong"
            reasoning = f"{etf1} significantly outperforming, strong style leadership"
        elif current_zscore < -2.0:
            signal = f"ROTATE TO {etf1}"
            strength = "Strong"
            reasoning = f"{etf1} underperforming, expect style mean reversion"
        elif current_zscore > 2.5:
            signal = f"STRONG {etf2} DOMINANCE"
            strength = "Very Strong"
            reasoning = f"{etf2} significantly outperforming, strong style leadership"
        elif current_zscore > 2.0:
            signal = f"ROTATE TO {etf2}"
            strength = "Strong"
            reasoning = f"{etf2} underperforming, expect style mean reversion"
        elif abs(current_zscore) < 0.5:
            signal = "NEUTRAL - Balanced Style Allocation"
            strength = "Neutral"
            reasoning = "Style factors balanced, no clear advantage"
        else:
            signal = "HOLD CURRENT STYLE ALLOCATION"
            strength = "Moderate"
            reasoning = "Maintain current style positioning"
        
        # Add economic context
        rate_context = ""
        if rate_environment != 0:
            if rate_environment > 0.02:
                rate_context = "Rising Rates (Favor Value/Small Cap)"
            elif rate_environment < -0.02:
                rate_context = "Falling Rates (Favor Growth)"
            else:
                rate_context = "Neutral Rate Environment"
        
        stress_context = ""
        if market_stress != 0:
            if market_stress > 25:
                stress_context = "High Stress (Favor Quality/Large Cap)"
            elif market_stress < 15:
                stress_context = "Low Stress (Favor Growth/Small Cap)"
            else:
                stress_context = "Moderate Stress"
        
        current_signals.append({
            'strategy': strategy_name,
            'signal': signal,
            'strength': strength,
            'z_score': current_zscore,
            'current_allocation': current_allocation,
            'reasoning': reasoning,
            'rate_context': rate_context,
            'stress_context': stress_context,
            'cycle_persistence': result['cycle_persistence'],
            'sharpe_ratio': result['sharpe_ratio']
        })
        
        print(f"{strategy_name}:")
        print(f"   Signal: {signal}")
        print(f"   Strength: {strength} (Z-Score: {current_zscore:.2f})")
        if rate_context:
            print(f"   Rate Context: {rate_context}")
        if stress_context:
            print(f"   Stress Context: {stress_context}")
        print(f"   Reasoning: {reasoning}")
        print(f"   Cycle Persistence: {result['cycle_persistence']:.1f} days")
        print(f"   Historical Sharpe: {result['sharpe_ratio']:.3f}")
        print()
    
    return pd.DataFrame(current_signals)

def analyze_style_market_context(style_prices):
    """Analyze broader market context for style rotation"""
    print("📊 Style Market Context Analysis:\n")
    
    # Interest rate analysis
    if 'TLT' in style_prices.columns:
        tlt_change_1m = style_prices['TLT'].pct_change(20).iloc[-1]
        tlt_change_3m = style_prices['TLT'].pct_change(60).iloc[-1]
        
        print(f"📈 Interest Rate Environment (TLT):")
        print(f"   1-month TLT change: {tlt_change_1m:.2%}")
        print(f"   3-month TLT change: {tlt_change_3m:.2%}")
        
        if tlt_change_3m < -0.05:  # TLT falling = rates rising
            rate_regime = "Rising Rate Environment - Favor Value over Growth, Financials benefit"
        elif tlt_change_3m > 0.05:  # TLT rising = rates falling
            rate_regime = "Falling Rate Environment - Favor Growth over Value, Tech benefits"
        else:
            rate_regime = "Neutral Rate Environment - Mixed style signals"
        
        print(f"   Rate Regime: {rate_regime}")
    
    # Volatility/Risk analysis
    if 'VIX' in style_prices.columns:
        current_vix = style_prices['VIX'].iloc[-1]
        vix_ma = style_prices['VIX'].rolling(30).mean().iloc[-1]
        
        print(f"\n📊 Risk Environment (VIX):")
        print(f"   Current VIX: {current_vix:.1f}")
        print(f"   30-day VIX average: {vix_ma:.1f}")
        
        if current_vix < 15:
            risk_regime = "Low Volatility - Favor Growth/Small Cap, Risk-on environment"
        elif current_vix > 25:
            risk_regime = "High Volatility - Favor Value/Large Cap/Quality, Risk-off environment"
        else:
            risk_regime = "Moderate Volatility - Balanced style environment"
        
        print(f"   Risk Regime: {risk_regime}")
    
    # Style performance comparison
    style_etfs = ['IWF', 'IWD', 'IWM', 'SPY']
    available_styles = [etf for etf in style_etfs if etf in style_prices.columns]
    
    if len(available_styles) >= 2:
        print(f"\n📈 Style Factor Performance (3-month):")
        for style in available_styles:
            style_return = style_prices[style].pct_change(60).iloc[-1]
            
            if style == 'IWF':
                style_name = "Growth (Russell 1000)"
            elif style == 'IWD':
                style_name = "Value (Russell 1000)"
            elif style == 'IWM':
                style_name = "Small Cap (Russell 2000)"
            elif style == 'SPY':
                style_name = "Large Cap (S&P 500)"
            else:
                style_name = style
            
            print(f"   {style_name}: {style_return:.2%}")

# Get current style signals
if 'style_rotation_results' in locals() and style_rotation_results:
    current_style_signals = get_current_style_signals(style_rotation_results)
    
    print(f"📊 Current Style Signal Summary:")
    display(current_style_signals[['strategy', 'signal', 'strength', 'rate_context', 'z_score', 'cycle_persistence', 'sharpe_ratio']].round(3))
    
    # Analyze market context
    analyze_style_market_context(style_prices)
    
    # Active style rotation opportunities
    active_style_signals = current_style_signals[
        (current_style_signals['signal'].str.contains('ROTATE|DOMINANCE')) & 
        (~current_style_signals['signal'].str.contains('HOLD'))
    ]
    
    if not active_style_signals.empty:
        print(f"\n🚨 Active Style Rotation Opportunities:")
        for _, signal in active_style_signals.iterrows():
            print(f"   {signal['strategy']}:")
            print(f"     {signal['signal']} ({signal['strength']})")
            if signal['rate_context']:
                print(f"     Rate: {signal['rate_context']}")
            if signal['stress_context']:
                print(f"     Stress: {signal['stress_context']}")
            print(f"     Z-Score: {signal['z_score']:.2f}")
            print(f"     Expected Persistence: {signal['cycle_persistence']:.1f} days")
            print(f"     {signal['reasoning']}")
    else:
        print(f"\n⏳ No active style rotation signals - Maintain current style allocations")
        
else:
    print("❌ No style-based rotation analysis available")

## 7. Style Strategy Summary

In [ ]:
print("📋 Style-Based Rotation Strategy Summary")
print("=" * 60)

if 'style_rotation_results' in locals() and style_rotation_results:
    print(f"\n🔬 Style Strategy Performance Analysis:")
    total_strategies = len(style_rotation_results)
    profitable_strategies = sum(1 for r in style_rotation_results if r['outperformance'] > 0)
    avg_outperformance = np.mean([r['outperformance'] for r in style_rotation_results])
    best_sharpe = max(r['sharpe_ratio'] for r in style_rotation_results)
    avg_persistence = np.mean([r['cycle_persistence'] for r in style_rotation_results])
    
    print(f"   Style strategies tested: {total_strategies}")
    print(f"   Outperforming strategies: {profitable_strategies}")
    print(f"   Success rate: {profitable_strategies/total_strategies:.1%}")
    print(f"   Average outperformance: {avg_outperformance:.2%}")
    print(f"   Best Sharpe ratio: {best_sharpe:.3f}")
    print(f"   Average cycle persistence: {avg_persistence:.1f} days")

print(f"\n💡 Style-Based Rotation Insights:")
print(f"   • Growth vs Value cycles driven by interest rates and risk appetite")
print(f"   • Small vs Large cap rotation follows economic cycle phases")
print(f"   • Style cycles tend to be persistent (weeks to months)")
print(f"   • Rate environment strongly influences Growth/Value performance")
print(f"   • Market stress drives flight to quality (Large Cap/Value)")

print(f"\n🎯 Style Rotation Advantages:")
print(f"   • Captures fundamental style factor premiums")
print(f"   • Benefits from behavioral biases and style momentum")
print(f"   • Enhanced by economic cycle timing")
print(f"   • Provides diversification within equity markets")
print(f"   • Long history of factor performance patterns")

print(f"\n🚀 Implementation Applications:")
print(f"   • Core satellite portfolio construction")
print(f"   • Factor timing overlays")
print(f"   • Risk parity strategy enhancements")
print(f"   • Economic cycle-based allocation models")
print(f"   • Style-neutral hedge fund strategies")

print(f"\n🌟 Key Style Strategy Components:")
print(f"   • Growth vs Value (IWF/IWD, VUG/VTV) - Rate and risk sensitivity")
print(f"   • Small vs Large Cap (IWM/SPY) - Economic cycle positioning")
print(f"   • Interest rate and volatility context integration")
print(f"   • Cycle persistence analysis for holding periods")
print(f"   • Economic regime-enhanced signal generation")

print(f"\n📊 Style Factor Characteristics:")
print(f"   • Growth: Sensitive to rates, momentum-driven, tech-heavy")
print(f"   • Value: Rate-resilient, contrarian, financial/industrial focus")
print(f"   • Small Cap: Economic sensitivity, higher volatility, early cycle")
print(f"   • Large Cap: Stability, late cycle, quality bias")
print(f"   • Style cycles typically last 3-18 months")

print(f"\n✅ Style-Based Rotation Strategy Complete")